Admission Dataset
Zulema Caldwell

Overview
Analyze college admission data to determine admission based on GRE, GPA, and rank


In [9]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()


Initializing Spark...
Spark found in :  /home/ubuntu/apps/spark
Spark config:
	 executor.memory=2g
	some_property=some_value
	spark.app.name=TestApp
	spark.master=local[*]
	spark.sql.warehouse.dir=/tmp/tmp0oo_tjz9
	spark.submit.deployMode=client
	spark.submit.pyFiles=
	spark.ui.showConsoleProgress=true
Spark UI running on port 4044


In [10]:
#import pandas as pd
#import numpy as np
#from wordsegment import load, segment
#load()

#dga = pd.read_csv('.csv')

admissionDF = spark.read.\
           option("header", "true").\
           option("inferSchema", "true").\
           csv("/data/college-admissions/admission-data.csv")
print(admissionDF)
admissionDF.printSchema()

DataFrame[_c0: int, site: string, source: string, label: string, toplevel: string, domain: string, words: string]
root
 |-- admit: integer (nullable = true)
 |-- gre: integer (nullable = true)
 |-- gpa: double (nullable = true)
 |-- rank: integer (nullable = true)



In [11]:
admissionDF.show()

+-----+---+----+----+
|admit|gre| gpa|rank|
+-----+---+----+----+
|    0|380|3.61|   3|
|    1|660|3.67|   3|
|    1|800| 4.0|   1|
|    0|640|3.19|   4|
|    0|520|2.93|   4|
|    1|760| 3.0|   2|
|    0|560|2.98|   1|
|    0|400|3.08|   2|
|    0|540|3.39|   3|
|    1|700|3.92|   2|
|    1|800| 4.0|   4|
|    0|440|3.22|   1|
|    1|760| 4.0|   1|
|    1|700|3.08|   2|
|    1|700| 4.0|   1|
|    0|480|3.44|   3|
|    1|780|3.87|   4|
|    0|360|2.56|   3|
|    1|800|3.75|   2|
|    0|540|3.81|   1|
+-----+---+----+----+
only showing top 20 rows



Create training and test dataset

In [22]:
admissionTrain, admissionTest = admissionDF.randomSplit([0.8, 0.2])
admissionTrain.show()
admissionTest.show()

+-----+---+----+----+
|admit|gre| gpa|rank|
+-----+---+----+----+
|    0|300|2.92|   4|
|    0|360|2.56|   3|
|    0|360|3.14|   1|
|    0|380|2.91|   4|
|    0|380|2.94|   3|
|    0|380|3.61|   3|
|    0|400|3.05|   2|
|    0|400|3.08|   2|
|    0|400|3.31|   3|
|    0|400|3.35|   3|
|    0|400|3.65|   2|
|    0|440|2.48|   4|
|    0|440|3.13|   4|
|    0|480|3.39|   4|
|    0|480|3.57|   2|
|    0|500|2.97|   4|
|    0|500|3.17|   3|
|    0|500|3.31|   3|
|    0|520| 2.9|   3|
|    0|520|2.93|   4|
+-----+---+----+----+
only showing top 20 rows

+-----+---+----+----+
|admit|gre| gpa|rank|
+-----+---+----+----+
|    0|440|3.22|   1|
|    0|480|3.44|   3|
|    0|500|2.71|   2|
|    0|520|3.29|   1|
|    0|540|3.78|   4|
|    0|560|3.32|   4|
|    0|600|3.48|   2|
|    0|640|3.19|   4|
|    0|700|2.88|   2|
|    1|500|3.13|   2|
|    1|620|3.18|   2|
|    1|680|3.85|   3|
|    1|800| 4.0|   1|
|    1|800| 4.0|   3|
+-----+---+----+----+



Vectorize 3 columns--GRE, GPA, RANK

In [24]:
from pyspark.ml.feature import VectorAssembler

## TODO : create featureVector with 'mpg' and 'cyl'
## Hint :  inputCols=['mpg', 'cyl']
assembler = VectorAssembler(inputCols=["gre", "gpa", "rank"], outputCol="features")
featureVector = assembler.transform(admissionTrain)
featureVector.show()

+-----+---+----+----+----------------+
|admit|gre| gpa|rank|        features|
+-----+---+----+----+----------------+
|    0|300|2.92|   4|[300.0,2.92,4.0]|
|    0|360|2.56|   3|[360.0,2.56,3.0]|
|    0|360|3.14|   1|[360.0,3.14,1.0]|
|    0|380|2.91|   4|[380.0,2.91,4.0]|
|    0|380|2.94|   3|[380.0,2.94,3.0]|
|    0|380|3.61|   3|[380.0,3.61,3.0]|
|    0|400|3.05|   2|[400.0,3.05,2.0]|
|    0|400|3.08|   2|[400.0,3.08,2.0]|
|    0|400|3.31|   3|[400.0,3.31,3.0]|
|    0|400|3.35|   3|[400.0,3.35,3.0]|
|    0|400|3.65|   2|[400.0,3.65,2.0]|
|    0|440|2.48|   4|[440.0,2.48,4.0]|
|    0|440|3.13|   4|[440.0,3.13,4.0]|
|    0|480|3.39|   4|[480.0,3.39,4.0]|
|    0|480|3.57|   2|[480.0,3.57,2.0]|
|    0|500|2.97|   4|[500.0,2.97,4.0]|
|    0|500|3.17|   3|[500.0,3.17,3.0]|
|    0|500|3.31|   3|[500.0,3.31,3.0]|
|    0|520| 2.9|   3| [520.0,2.9,3.0]|
|    0|520|2.93|   4|[520.0,2.93,4.0]|
+-----+---+----+----+----------------+
only showing top 20 rows



Do binary classification using LogisticRegression

In [25]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.1, elasticNetParam=0.8, labelCol='admit')

#Fit the Model
lrModel = lr.fit(featureVector)

In [26]:
lrModel.transform(assembler.transform(admissionTest)).show()

+-----+---+----+----+----------------+--------------------+--------------------+----------+
|admit|gre| gpa|rank|        features|       rawPrediction|         probability|prediction|
+-----+---+----+----+----------------+--------------------+--------------------+----------+
|    0|440|3.22|   1|[440.0,3.22,1.0]|[0.00763001634508...|[0.50190749483220...|       0.0|
|    0|480|3.44|   3|[480.0,3.44,3.0]|[0.50692614322887...|[0.62408561470433...|       0.0|
|    0|500|2.71|   2|[500.0,2.71,2.0]|[0.14345894427855...|[0.53580335297267...|       0.0|
|    0|520|3.29|   1|[520.0,3.29,1.0]|[-0.1349883684976...|[0.46630405923494...|       1.0|
|    0|540|3.78|   4|[540.0,3.78,4.0]|[0.69792064616081...|[0.66772659153509...|       0.0|
|    0|560|3.32|   4|[560.0,3.32,4.0]|[0.63127597693628...|[0.65277872976616...|       0.0|
|    0|600|3.48|   2|[600.0,3.48,2.0]|[0.00948067491505...|[0.50237015097572...|       0.0|
|    0|640|3.19|   4|[640.0,3.19,4.0]|[0.47567745679977...|[0.61672665095495...|

In [32]:
#Accuracy Score
trainingSummary = lrModel.summary

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.7093023255813954
FPR: 0.3123725010199918
TPR: 0.7093023255813953
F-measure: 0.706261978965666
Precision: 0.7081388700519899
Recall: 0.7093023255813953


In [ ]:
Define WSSSE

In [41]:
import numpy as np

def calculate_wsse (predictions, model):
    centers = model.clusterCenters()
    
    wssse = 0
    for i, row in enumerate(predictions.collect()):
        feature = row['features']
        prediction = row['prediction']
        center = centers[prediction]
        diff = feature - center
        error = np.sqrt(sum([x**2 for x in diff]))
        wssse += error
        # print ('row={}, feature : {} , prediction : {} , center : {},  diff : {}, error : {},  wssse={}'.
        #      format(i, feature, prediction, center, diff, error, wssse))
        
    return wssse



Try KMeans with WSSSE

In [46]:

%%time 

from pyspark.ml.clustering import KMeans

k = 2
kmeans = KMeans().setK(k).setMaxIter(10)
model = kmeans.fit(featureVector)


CPU times: user 7.6 ms, sys: 1.93 ms, total: 9.53 ms
Wall time: 1.51 s


In [47]:
predictions = model.transform(assembler.transform(admissionTest))
predictions.orderBy(['prediction', 'rank']).show()

+-----+---+----+----+----------------+----------+
|admit|gre| gpa|rank|        features|prediction|
+-----+---+----+----+----------------+----------+
|    0|520|3.29|   1|[520.0,3.29,1.0]|         0|
|    0|440|3.22|   1|[440.0,3.22,1.0]|         0|
|    0|500|2.71|   2|[500.0,2.71,2.0]|         0|
|    1|500|3.13|   2|[500.0,3.13,2.0]|         0|
|    0|480|3.44|   3|[480.0,3.44,3.0]|         0|
|    0|540|3.78|   4|[540.0,3.78,4.0]|         0|
|    0|560|3.32|   4|[560.0,3.32,4.0]|         0|
|    1|800| 4.0|   1| [800.0,4.0,1.0]|         1|
|    0|700|2.88|   2|[700.0,2.88,2.0]|         1|
|    1|620|3.18|   2|[620.0,3.18,2.0]|         1|
|    0|600|3.48|   2|[600.0,3.48,2.0]|         1|
|    1|680|3.85|   3|[680.0,3.85,3.0]|         1|
|    1|800| 4.0|   3| [800.0,4.0,3.0]|         1|
|    0|640|3.19|   4|[640.0,3.19,4.0]|         1|
+-----+---+----+----+----------------+----------+



In [48]:
from pyspark.ml.evaluation import ClusteringEvaluator

wssse = calculate_wsse(predictions, model)

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print ("k={}, wssse={},  silhouette_score={}".format(k, wssse, silhouette))

k=2, wssse=694.7342494034673,  silhouette_score=0.7004089392198949


It appears that LogisticRegression is a better prediction tool than KMeans for this dataset.